### import

In [7]:
import pandas as pd
import numpy as np

from pymoo.indicators.hv import HV
from pymoo.indicators.igd import IGD

### 读取文件

In [8]:
'''None Dominated Solutions'''
def find_non_dominated_points(points):
    # 输入: points 是一个二维 numpy 数组，形状为 (N, 2)，其中 N 是点的数量
    n_points = points.shape[0]
    is_dominated = np.zeros(n_points, dtype=bool)  # 初始化一个数组，用来标记是否被支配

    for i in range(n_points):
        for j in range(n_points):
            if i != j:
                # 如果点 j 在所有目标上都优于点 i，则 i 被支配
                if (points[j][0] >= points[i][0] and points[j][1] >= points[i][1]) and \
                   (points[j][0] > points[i][0] or points[j][1] > points[i][1]):
                    is_dominated[i] = True
                    break  # 一旦发现被支配，无需再检查

    # 返回非支配点
    return points[~is_dominated]

# # 示例：调用函数并传入文件路径
# # file_path_ms = 'NSL-KDD Generator_MaxSample'  # NSL Gen HV
# # file_path_ms = 'NSL-KDD Discriminator_MaxSample.csv'  # NSL Dis HV
# # file_path_ms = 'UNSW-NB15 Generator_MaxSample.csv'  # UNSW Gen HV
# file_path_ms = 'UNSW-NB15 Discriminator_MaxSample.csv'  # UNSW Dis HV
# max_sampling_all = read_coordinates(file_path_ms)
# max_sampling_all = np.array(max_sampling_all)
# # 筛选非支配点
# max_sampling = find_non_dominated_points(max_sampling_all)

# 示例：调用函数并传入文件路径
# file_path_DMS = 'NSL-KDD Discriminator_DoubleMutationSample.csv'  # NSL Dis None Dominated Solutions
# file_path_DMS = 'UNSW-NB15 Discriminator_DoubleMutationSample.csv'  # UNSW Dis None Dominated Solutions
# file_path_DMS = 'NSL-KDD Generator_DoubleMutationSample.csv'  # NSL Gen None Dominated Solutions
file_path_DMS = 'UNSW-NB15 Generator_DoubleMutationSample.csv'  # UNSW Gen None Dominated Solutions

double_mutation_sampling = pd.read_csv(file_path_DMS)
double_mutation_sampling = np.array(double_mutation_sampling)

double_mutation_sampling

array([[0.00148361, 0.21875   ],
       [0.00115833, 0.4140625 ],
       [0.00103476, 0.4375    ],
       [0.28910768, 0.0625    ],
       [0.28884982, 0.078125  ],
       [0.69090831, 0.03125   ],
       [0.68947941, 0.0390625 ],
       [0.68774308, 0.046875  ],
       [0.00128221, 0.3515625 ],
       [0.00515913, 0.1875    ],
       [0.00137341, 0.296875  ],
       [0.32835995, 0.0546875 ],
       [0.00146869, 0.2421875 ],
       [0.0014675 , 0.2578125 ],
       [0.00807917, 0.1328125 ],
       [0.00804481, 0.140625  ]])

### 去重

In [9]:
double_mutation_sampling = np.unique(double_mutation_sampling, axis=0)

# # # 对第一列取倒数
# print(f'Latency: {1 / double_mutation_sampling[:, 0]}\n')
# double_mutation_sampling[:, 0] = 1 / double_mutation_sampling[:, 0]

double_mutation_sampling

array([[0.00103476, 0.4375    ],
       [0.00115833, 0.4140625 ],
       [0.00128221, 0.3515625 ],
       [0.00137341, 0.296875  ],
       [0.0014675 , 0.2578125 ],
       [0.00146869, 0.2421875 ],
       [0.00148361, 0.21875   ],
       [0.00515913, 0.1875    ],
       [0.00804481, 0.140625  ],
       [0.00807917, 0.1328125 ],
       [0.28884982, 0.078125  ],
       [0.28910768, 0.0625    ],
       [0.32835995, 0.0546875 ],
       [0.68774308, 0.046875  ],
       [0.68947941, 0.0390625 ],
       [0.69090831, 0.03125   ]])

In [10]:
# # 计算HV

# # 定义解集及参考点
# reference_point = [0.11, 0.5]  # NSL Gen:[0.11, 0.5]; NSL Dis:[0.002, 0.8]
#                                # UNSW Gen:[xx, yy]; UNSW Dis:
# # 计算Max Sampling超体积
# hv_ms = HV(ref_point=reference_point)
# hypervolume_ms = hv_ms.do(max_sampling)
# print("Max Sampling Hypervolume:", hypervolume_ms)

# # 计算Double Mutation Sampling超体积
# hv_dms = HV(ref_point=reference_point)
# hypervolume_dms = hv_dms.do(double_mutation_sampling)
# print("Double Mutation Sampling Hypervolume:", hypervolume_dms)


##### NSL Gen HV    
##### Max Sampling 超体积: 0.03090679805876238
##### Double Mutation Sampling 超体积: 0.04248747039571762

### 熵权法求权重系数
1.标准化矩阵

In [11]:
max_obj1, min_obj1 = np.max(double_mutation_sampling[:, 0]), np.min(double_mutation_sampling[:, 0])
max_obj2, min_obj2 = np.max(double_mutation_sampling[:, 1]), np.min(double_mutation_sampling[:, 1])

max_values, min_values = np.array([max_obj1, max_obj2]), np.array([min_obj1, min_obj2])
forward_reverse_list = [True, True]
max_obj1, min_obj1, max_obj2, min_obj2

(0.6909083139314383, 0.0010347621582421, 0.4375, 0.03125)

2.计算比重矩阵 --> 信息熵 --> 权重系数

In [12]:
def entropy_weight(matrix_origin, max_values, min_values, forward_reverse_list): # forward:True; reverse:False
    matrix = np.copy(matrix_origin)
    information_entropy = []
    for obj in range(max_values.shape[0]):
        if forward_reverse_list[obj]:
            matrix[:, obj] = (matrix[:, obj] - min_values[obj]) / (max_values[obj] - min_values[obj])
        else:
            matrix[:, obj] = (max_values[obj] - matrix[:, obj]) / (max_values[obj] - min_values[obj])

        # weighted_matrix
        matrix[:, obj] = matrix[:, obj] / np.sum(matrix[:, obj])

        # information_entropy
        h = 0.0
        for col in matrix[:, obj]:
            if col != 0.0 :
                add = -col*np.log(col)
                h += add
            else:                
                h += 0.0
        information_entropy.append(h)

    # weights
    weights_sum = np.sum([np.log(matrix.shape[0]) - h for h in information_entropy])
    weights = [(np.log(matrix.shape[0]) - h) / weights_sum for h in information_entropy]
    return matrix, information_entropy, weights

matrix_weights, hs, weights = entropy_weight(matrix_origin=double_mutation_sampling, 
                                             max_values=max_values, 
                                             min_values=min_values, 
                                             forward_reverse_list=forward_reverse_list)

weights

[0.7182873396103959, 0.281712660389604]

### 模糊评价

In [7]:
def triangular_membership(rank, x, a, b, c):
    """
    计算三角形隶属度函数的隶属度值
    :param x: 输入值
    :param a: 左端点
    :param b: 顶点
    :param c: 右端点
    :return: 隶属度值
    """
    if rank == 1:
        if x <= a:
            return 1.0
        elif a < x <= b:
            return (b - x) / (b - a)
        elif x > b:
            return 0.0
    if rank == 2:
        if  a <= x <= b:
            return (x - a) / (b - a)
        elif b < x <= c:
            return (c - x) / (c - b)
        elif x < a or x > c:
            return 0.0
    if rank == 3:
        if x <= b:
            return 0.0
        elif b < x <= c:
            return (x - b) / (c - b)
        elif x > c:
            return 1.0

def calculate_membership(accuracy, a, b, c, reverse=False):
    """
    根据准确率计算其在“差”、“中”、“好”三个等级中的隶属度
    :rank: 评价等级序号
    :param accuracy: 输入的准确率值 (0-100)
    :return: 三个等级的隶属度
    """
    poor = triangular_membership(rank=1, x=accuracy, 
                                 a=a, b=b, c=c)
    medium = triangular_membership(rank=2, x=accuracy, 
                                 a=a, b=b, c=c)
    good = triangular_membership(rank=3, x=accuracy, 
                                 a=a, b=b, c=c)
    if reverse == False:
        return [poor, medium, good]
    else:
        return [good, medium, poor]

In [ ]:
# Dis Latency:   a=1000, b=2000, c=3000
# Dis Accuracy:  a=0.70,   b=0.75,   c=0.80

# Gen Diversity: a=800, b=1600, c=2400
# Gen Accuracy:  a=0.20,   b=0.30,   c=0.40

weights = [0.5, 0.5]
print(f'Weights:{weights}')
for obj1, obj2 in double_mutation_sampling:
    # For Discriminator
    membership_obj1 = calculate_membership(obj1, a=1000, b=2000, c=3000, reverse=True)
    membership_obj2 = calculate_membership(obj2, a=0.70, b=0.75, c=0.80, reverse=False)

    # # For Generator
    # membership_obj1 = calculate_membership(obj1, a=0.001, b=0.01, c=0.1, reverse=False)
    # membership_obj2 = calculate_membership(obj2, a=0.1, b=0.20, c=0.40, reverse=False)

    membership_matrix = np.array([membership_obj1, membership_obj2])
    weights = np.array(weights).reshape(1, -1)

    assessment_matrix = np.dot(weights, membership_matrix).flatten()
    assessment_value = np.sum([0.3333, 0.6667, 1] * assessment_matrix)

    print(f'Point:[{obj1}, {obj2}]:\n      {membership_obj1}\n      {membership_obj2}\nFinal:{assessment_value}\n')

Weights:[0.5, 0.5]
Point:[0.00040277, 0.8125]:
      [0.0, 0.0, 1.0]
      [0.0, 0.0, 1.0]
Final:1.0

Point:[0.000443054, 0.80469]:
      [0.0, 0.0, 1.0]
      [0.0, 0.0, 1.0]
Final:1.0

Point:[0.000597865, 0.79688]:
      [0.0, 0.0, 1.0]
      [0.0, 0.06240000000000018, 0.9375999999999998]
Final:0.9896010399999999

Point:[0.0018, 0.75781]:
      [0.0, 0.0, 1.0]
      [0.0, 0.8438000000000004, 0.15619999999999953]
Final:0.8593807299999998

Point:[0.1617, 0.74219]:
      [0.0, 0.0, 1.0]
      [0.15619999999999953, 0.8438000000000004, 0.0]
Final:0.8073114600000001

